<a href="https://colab.research.google.com/github/AlvinChiew/PythonBasics/blob/main/PyDataClasses_Pydantic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b>Objective</b> : use Pydantic to validate selection defined by data classes

* Pydantic will prompt error when object has attribute not in predefined options

# Imports & Classes

In [1]:
!pip install pydantic

     |████████████████████████████████| 9.2MB 5.2MB/s 


In [2]:
from typing import Tuple
from enum import Enum 

In [3]:
class Flavor(str, Enum):
    chocolate = 'chocolate'
    vanilla = 'vanilla'
    mint = 'mint'
    mocha = 'mocha'

class Topping(str, Enum):
    sprinkles = 'sprinkles'
    chocolate_chips = 'chocolate chips'
    whipped_cream = 'whipped cream'
    cookies = 'cookies'
    hot_fudge = 'hot fudge'

# Model : dataclass

In [4]:
from pydantic.dataclasses import dataclass

In [5]:
@dataclass  # decorator
class IceCreamMix:
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int

## Invalid Attribute

In [6]:
### Validation will prompt invalid value which falls out of defined class ###

def with_invalid_topping(): 
    ice_cream_mix = IceCreamMix(
        "Best Seller",
        Flavor.mocha,
        (Topping.sprinkles, 'InvalidTopping'),
        2
    )
    print(ice_cream_mix)

with_invalid_topping()

ValidationError: ignored

## Valid Attribute

In [7]:
def with_valid_topping():
    ice_cream_mix = IceCreamMix(
        "Best Seller",
        Flavor.mocha,
        (Topping.sprinkles, Topping.chocolate_chips),
        '2'     # Pydantic will also auto-fix simple type error
    )
    print(ice_cream_mix)

with_valid_topping()

IceCreamMix(name='Best Seller', flavor=<Flavor.mocha: 'mocha'>, toppings=(<Topping.sprinkles: 'sprinkles'>, <Topping.chocolate_chips: 'chocolate chips'>), scoops=2)


# Model : BaseModel

In [8]:
from pydantic import BaseModel, ValidationError, Field, validator, root_validator

# Extra feature : serialization, Json output for object and errors/exceptions, define attribute value in range 

In [9]:
class IceCreamMix(BaseModel):     # decorator
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int

## Valid Attribute

In [10]:
def with_valid_topping(): 
    ice_cream_mix = IceCreamMix(
        name = "Best Seller",
        flavor = Flavor.mocha,
        toppings = (Topping.sprinkles, Topping.chocolate_chips),
        scoops = 2
    )
    print(ice_cream_mix.json())     # Output result as JSON

with_valid_topping()

{"name": "Best Seller", "flavor": "mocha", "toppings": ["sprinkles", "chocolate chips"], "scoops": 2}


## Invalid Attribute

In [11]:
def with_invalid_flavor(): 
    try:
        ice_cream_mix = IceCreamMix(
            name = "Best Seller",
            flavor = 'RandomFlavor',
            toppings = (Topping.sprinkles, Topping.chocolate_chips),
            scoops = 2
        )
    except ValidationError as err:
        print(err.json())       # Output error as JSON

    print(ice_cream_mix.json())     # Output result as JSON

with_invalid_flavor()

[
  {
    "loc": [
      "flavor"
    ],
    "msg": "value is not a valid enumeration member; permitted: 'chocolate', 'vanilla', 'mint', 'mocha'",
    "type": "type_error.enum",
    "ctx": {
      "enum_values": [
        "chocolate",
        "vanilla",
        "mint",
        "mocha"
      ]
    }
  }
]


UnboundLocalError: ignored

## Define Attribute Value in range

In [12]:
class IceCreamMix(BaseModel):     # decorator
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    scoops: int = Field (..., gt=0, lt=5)       # scoops must be >0 and <5; ... = any value, can be replaced with default value, e.g. 2

    @validator('toppings')
    def check_toppings(cls, toppings):          # topping must not >2
        if len(toppings) > 2 :
            raise ValueError('Too many toppings')
        return toppings

### Invalid \# scoops - restrict attribute (int) to a range

In [13]:
def with_invalid_scoops(): 
    try:
        ice_cream_mix = IceCreamMix(
            name = "Best Seller",
            flavor = Flavor.mocha,
            toppings = (Topping.sprinkles, Topping.chocolate_chips),
            scoops = 5
        )
    except ValidationError as err:
        print(err.json())       # Output error as JSON

    print(ice_cream_mix.json())     # Output result as JSON

with_invalid_scoops()

[
  {
    "loc": [
      "scoops"
    ],
    "msg": "ensure this value is less than 5",
    "type": "value_error.number.not_lt",
    "ctx": {
      "limit_value": 5
    }
  }
]


UnboundLocalError: ignored

### Invalid \# toppings - restrict \# values in attribute

In [14]:
def with_invalid_scoops(): 
    try:
        ice_cream_mix = IceCreamMix(
            name = "Best Seller",
            flavor = Flavor.mocha,
            toppings = (Topping.sprinkles, Topping.chocolate_chips, Topping.whipped_cream),
            scoops = 2
        )
    except ValidationError as err:
        print(err.json())       # Output error as JSON

    print(ice_cream_mix.json())     # Output result as JSON

with_invalid_scoops()

[
  {
    "loc": [
      "toppings"
    ],
    "msg": "Too many toppings",
    "type": "value_error"
  }
]


UnboundLocalError: ignored

### Invalid container - restrict attribute value based on other attribute

In [20]:
class Container(str, Enum):
    cup = 'cup',
    cone = 'cone',
    waffle = 'waffle'

class IceCreamMix2(BaseModel):     # decorator
    name: str
    flavor: Flavor
    toppings: Tuple[Topping, ...]
    container : Container
    scoops: int

    @root_validator
    def check_toppings(cls, values):
        container = values.get('container')
        toppings = values.get('toppings')
        if container == Container.cone:
            if Topping.hot_fudge in toppings:
                raise ValueError('cone cannot contain hot fudge')
        return values

In [21]:
def with_invalid_container(): 
    try:
        ice_cream_mix2 = IceCreamMix2(
            name = "Best Seller",
            flavor = Flavor.mocha,
            toppings = (Topping.chocolate_chips, Topping.hot_fudge),
            container = Container.cone,
            scoops = 2
        )
    except ValidationError as err:
        print(err.json())       # Output error as JSON

    print(ice_cream_mix2.json())     # Output result as JSON

with_invalid_container()

[
  {
    "loc": [
      "__root__"
    ],
    "msg": "cone cannot contain hot fudge",
    "type": "value_error"
  }
]


UnboundLocalError: ignored